# TODO

- you need plots


In [1]:
import numpy as np
import pandas as pd
import pickle
import statsmodels.api as sm
from linearmodels.panel import PanelOLS


In [2]:
data_dir = '/home/simon/Documents/Bodies/data/OD_dataframes_compiled/'

# with open(f'{data_dir}bodies_df_fatal.pkl', 'rb') as file:
#     bodies_df_fatal = pickle.load(file)

with open(f'{data_dir}bodies_df_exposure.pkl', 'rb') as file:
    bodies_df_exposure = pickle.load(file)

# NOTES:


RQ:
- To what extent does the distribution of content in the images, differs across publication status.
 
- To What extent does the distribution of content in the images corrolate with the magnitude/exposure of conflict 


In [3]:
# pre

list_of_features_mean = [feature for feature in bodies_df_exposure.columns if 'mean' in feature] 

df_sub = bodies_df_exposure[list_of_features_mean + ['log_best', 'tce', 'tsce', 'publication', 'month_id', 'gid']].copy()
df_sub = df_sub[df_sub['person_mean'] != 0] # no people, no entry
df_sub[['Published', 'Raw', 'Submitted']] = pd.get_dummies(df_sub[['publication']])
df_sub.drop(columns= 'publication', inplace= True)
df_sub.dropna(inplace= True)
df_sub.set_index(['gid', 'month_id'], inplace = True)
df_sub['intPubBest'] = df_sub['Published'] * df_sub['log_best']
df_sub['intPubTce'] = df_sub['Published'] * df_sub['tce']
df_sub['intPubTsce'] = df_sub['Published'] * df_sub['tsce']


# also drop all with no persons in them

# gender
## Contiues dependents:

$$
female_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$

Interestingly, the results holds is we have male mean as dependent. I.e. then Published is significantly negativly corrolated with published 


In [4]:
X = df_sub.loc[:,['log_best', 'Published', 'Submitted', 'person_mean']] # raw is ref
y = df_sub.loc[:, 'female_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed = True)

# m = PanelOLS.from_formula('female_mean ~ log_best + Published + Submitted + person_mean + EntityEffects + TimeEffects', data=df_sub)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,female_mean,R-squared:,0.2542
Estimator:,PanelOLS,R-squared (Between):,0.7204
No. Observations:,126548,R-squared (Within):,0.2526
Date:,"Thu, Jun 30 2022",R-squared (Overall):,0.4840
Time:,10:24:51,Log-likelihood,-1.454e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.078e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-2.909e+18


$$
male_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} tce_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$


In [5]:
X = df_sub.loc[:,['log_best', 'Published', 'Submitted', 'person_mean']] # raw is ref
y = df_sub.loc[:, 'male_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed = True)

# m = PanelOLS.from_formula('female_mean ~ log_best + Published + Submitted + person_mean + EntityEffects + TimeEffects', data=df_sub)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,male_mean,R-squared:,0.8761
Estimator:,PanelOLS,R-squared (Between):,0.9647
No. Observations:,126548,R-squared (Within):,0.8762
Date:,"Thu, Jun 30 2022",R-squared (Overall):,0.9278
Time:,10:25:24,Log-likelihood,-1.633e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,2.235e+05
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,9.185e+18



$$
female_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} tce_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$


In [6]:
X = df_sub.loc[:,['tce', 'Published', 'Submitted', 'person_mean']] # raw is ref
y = df_sub.loc[:, 'female_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed = True)

# m = PanelOLS.from_formula('female_mean ~ log_best + Published + Submitted + person_mean + EntityEffects + TimeEffects', data=df_sub)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,female_mean,R-squared:,0.2545
Estimator:,PanelOLS,R-squared (Between):,0.7144
No. Observations:,126548,R-squared (Within):,0.2626
Date:,"Thu, Jun 30 2022",R-squared (Overall):,0.5319
Time:,10:26:08,Log-likelihood,-1.453e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.079e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,8.728e+18


In [7]:
X = df_sub.loc[:,['tce', 'Published', 'Submitted', 'person_mean']] # raw is ref
y = df_sub.loc[:, 'male_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed = True)

# m = PanelOLS.from_formula('female_mean ~ log_best + Published + Submitted + person_mean + EntityEffects + TimeEffects', data=df_sub)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,male_mean,R-squared:,0.8761
Estimator:,PanelOLS,R-squared (Between):,0.9599
No. Observations:,126548,R-squared (Within):,0.8762
Date:,"Thu, Jun 30 2022",R-squared (Overall):,0.9178
Time:,10:26:27,Log-likelihood,-1.633e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,2.235e+05
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,3.062e+18



$$
female_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} tsce_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$


In [11]:
X = df_sub.loc[:,['tsce', 'Published', 'Submitted', 'person_mean']] # raw is ref
y = df_sub.loc[:, 'female_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed = True)

# m = PanelOLS.from_formula('female_mean ~ log_best + Published + Submitted + person_mean + EntityEffects + TimeEffects', data=df_sub)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,female_mean,R-squared:,0.2545
Estimator:,PanelOLS,R-squared (Between):,0.7172
No. Observations:,126548,R-squared (Within):,0.2608
Date:,"Wed, Jun 29 2022",R-squared (Overall):,0.5188
Time:,10:45:46,Log-likelihood,-1.453e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.079e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,2.182e+18



$$
female_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} tsce_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$


In [9]:
X = df_sub.loc[:,['tsce', 'Published', 'Submitted', 'person_mean']] # raw is ref
y = df_sub.loc[:, 'male_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed = True)

# m = PanelOLS.from_formula('female_mean ~ log_best + Published + Submitted + person_mean + EntityEffects + TimeEffects', data=df_sub)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,male_mean,R-squared:,0.8761
Estimator:,PanelOLS,R-squared (Between):,0.9593
No. Observations:,126548,R-squared (Within):,0.8761
Date:,"Thu, Jun 30 2022",R-squared (Overall):,0.9168
Time:,10:27:21,Log-likelihood,-1.633e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,2.235e+05
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,4.593e+18


$$
religiousGarments_{it} =  \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \beta_{5} female_{it} + \epsilon_{it}  
$$

In [8]:
X = df_sub.loc[:,['female_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3404
Estimator:,PanelOLS,R-squared (Between):,0.9104
No. Observations:,126548,R-squared (Within):,0.3517
Date:,"Wed, Jun 29 2022",R-squared (Overall):,0.4274
Time:,10:43:54,Log-likelihood,-1.74e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.632e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,4.465e+19


$$
religiousGarments_{it} =  \alpha_{i} + \gamma_{t} + \beta_{1} tce_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \beta_{5} female_{it} + \epsilon_{it}  
$$

In [14]:
X = df_sub.loc[:,['female_mean', 'tce', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3407
Estimator:,PanelOLS,R-squared (Between):,0.8904
No. Observations:,126548,R-squared (Within):,0.3499
Date:,"Wed, Jun 29 2022",R-squared (Overall):,0.3604
Time:,10:47:25,Log-likelihood,-1.738e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.634e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-5.894e+20


$$
religiousGarments_{it} =  \alpha_{i} + \gamma_{t} + \beta_{1} tsce_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \beta_{5} female_{it} + \epsilon_{it}  
$$

In [13]:
X = df_sub.loc[:,['female_mean', 'tsce', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3406
Estimator:,PanelOLS,R-squared (Between):,0.8770
No. Observations:,126548,R-squared (Within):,0.3481
Date:,"Wed, Jun 29 2022",R-squared (Overall):,0.3463
Time:,10:47:22,Log-likelihood,-1.738e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.633e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-5.351e+19


# with interactions

In [18]:
X = df_sub.loc[:,['log_best', 'intPubBest', 'Published', 'Submitted', 'person_mean']] # raw is ref
y = df_sub.loc[:, 'female_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed = True)

# m = PanelOLS.from_formula('female_mean ~ log_best + Published + Submitted + person_mean + EntityEffects + TimeEffects', data=df_sub)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

/tmp/ipykernel_20643/2099713059.py:12: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Published

  m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)


Dep. Variable:,female_mean,R-squared:,0.2542
Estimator:,PanelOLS,R-squared (Between):,0.7204
No. Observations:,126548,R-squared (Within):,0.2526
Date:,"Wed, Jun 29 2022",R-squared (Overall):,0.4839
Time:,10:51:53,Log-likelihood,-1.454e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.078e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-2.099e+18


In [19]:
X = df_sub.loc[:,['tce', 'intPubTce', 'Published', 'Submitted', 'person_mean']] # raw is ref
y = df_sub.loc[:, 'female_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed = True)

# m = PanelOLS.from_formula('female_mean ~ log_best + Published + Submitted + person_mean + EntityEffects + TimeEffects', data=df_sub)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

/tmp/ipykernel_20643/1668388244.py:12: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Published

  m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)


Dep. Variable:,female_mean,R-squared:,0.2545
Estimator:,PanelOLS,R-squared (Between):,0.7144
No. Observations:,126548,R-squared (Within):,0.2626
Date:,"Wed, Jun 29 2022",R-squared (Overall):,0.5319
Time:,10:52:20,Log-likelihood,-1.453e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.079e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-4.549e+18


In [20]:
X = df_sub.loc[:,['tsce', 'intPubTsce', 'Published', 'Submitted', 'person_mean']] # raw is ref
y = df_sub.loc[:, 'female_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed = True)

# m = PanelOLS.from_formula('female_mean ~ log_best + Published + Submitted + person_mean + EntityEffects + TimeEffects', data=df_sub)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

/tmp/ipykernel_20643/3023179014.py:12: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Published

  m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)


Dep. Variable:,female_mean,R-squared:,0.2545
Estimator:,PanelOLS,R-squared (Between):,0.7172
No. Observations:,126548,R-squared (Within):,0.2608
Date:,"Wed, Jun 29 2022",R-squared (Overall):,0.5188
Time:,10:52:49,Log-likelihood,-1.453e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,1.079e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-8.416e+18


In [21]:
X = df_sub.loc[:,['female_mean', 'log_best', 'intPubBest', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

/tmp/ipykernel_20643/2186205462.py:10: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Published

  m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)


Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3404
Estimator:,PanelOLS,R-squared (Between):,0.9104
No. Observations:,126548,R-squared (Within):,0.3517
Date:,"Wed, Jun 29 2022",R-squared (Overall):,0.4274
Time:,10:54:40,Log-likelihood,-1.74e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.632e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-3.426e+19


In [22]:
X = df_sub.loc[:,['female_mean', 'tce', 'intPubTce', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

/tmp/ipykernel_20643/1318747554.py:10: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Published

  m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)


Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3407
Estimator:,PanelOLS,R-squared (Between):,0.8904
No. Observations:,126548,R-squared (Within):,0.3499
Date:,"Wed, Jun 29 2022",R-squared (Overall):,0.3604
Time:,10:54:43,Log-likelihood,-1.738e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.634e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-4.031e+20


In [23]:
X = df_sub.loc[:,['female_mean', 'tsce', 'intPubTsce', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True,
            drop_absorbed=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

/tmp/ipykernel_20643/3981241453.py:10: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Published

  m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)


Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3406
Estimator:,PanelOLS,R-squared (Between):,0.8770
No. Observations:,126548,R-squared (Within):,0.3481
Date:,"Wed, Jun 29 2022",R-squared (Overall):,0.3463
Time:,10:54:48,Log-likelihood,-1.738e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.633e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,4.24e+19


### Binary dependent (linar probability model)

$$
binaryFemale_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$

$$
binaryFeligiousGarments_{it} =  \gamma_{i} + \alpha_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \beta_{5} female_{it} + \epsilon_{it}  
$$

In [10]:
X = df_sub.loc[:,['log_best', 'person_mean', 'Published', 'Submitted']] # raw is ref
y = (df_sub.loc[:, 'female_mean'] > 0).astype(int)


m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,female_mean,R-squared:,0.0758
Estimator:,PanelOLS,R-squared (Between):,0.4731
No. Observations:,126548,R-squared (Within):,0.0772
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.2993
Time:,11:55:29,Log-likelihood,-8.038e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,2593.7
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-2.646e+17


In [11]:
X = df_sub.loc[:,['female_mean', 'person_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = (df_sub.loc[:, 'religiousGarmentFemale_mean'] > 0).astype(int)

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.2533
Estimator:,PanelOLS,R-squared (Between):,0.8558
No. Observations:,126548,R-squared (Within):,0.2663
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4119
Time:,11:55:31,Log-likelihood,-4.076e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,8579.7
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(5,126473)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,2.445e+20


## Logged dependent

$$
logFemale_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$

$$
logFeligiousGarments_{it} =  \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \beta_{5} female_{it} + \epsilon_{it}  
$$

In [12]:
X = df_sub.loc[:,['log_best', 'person_mean', 'Published', 'Submitted']] # raw is ref
y = np.log(df_sub.loc[:, 'female_mean']+1)


m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,female_mean,R-squared:,0.2046
Estimator:,PanelOLS,R-squared (Between):,0.7023
No. Observations:,126548,R-squared (Within):,0.2032
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4531
Time:,11:55:33,Log-likelihood,-5.737e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,8133.4
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,7.965e+19


In [13]:
X = df_sub.loc[:,['female_mean', 'person_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = np.log(df_sub.loc[:, 'religiousGarmentFemale_mean'] + 1)

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel/results.py:87: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")
/home/simon/anaconda3/envs/scikit_env/lib/python3.8/site-packages/linearmodels/panel

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3339
Estimator:,PanelOLS,R-squared (Between):,0.9286
No. Observations:,126548,R-squared (Within):,0.3463
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4581
Time:,11:55:35,Log-likelihood,3.722e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,1.268e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(5,126473)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,9.003e+19


## ratio dependent

$$
ratioFemalePerson_{it} = \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \epsilon_{it}   
$$

$$
ratioFemaleReligiousGarments_{it} =  \alpha_{i} + \gamma_{t} + \beta_{1} logConflict_{it} + \beta_{2} published_{it} + \beta_{3} submitted_{it} + \beta_{4} persons_{it} + \beta_{5} + female_{it} \epsilon_{it}  
$$

In [14]:
X = df_sub.loc[:,['log_best', 'person_mean', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'female_mean']/df_sub.loc[:, 'person_mean']


m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,0,R-squared:,0.0070
Estimator:,PanelOLS,R-squared (Between):,-0.1795
No. Observations:,126548,R-squared (Within):,0.0074
Date:,"Mon, Jun 27 2022",R-squared (Overall):,-0.0866
Time:,11:55:37,Log-likelihood,-4.297e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,222.22
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-9.783e+18


In [15]:
df_sub_fem = df_sub[df_sub['female_mean']>0].copy()

X = df_sub_fem.loc[:,['female_mean', 'person_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = df_sub_fem.loc[:, 'religiousGarmentFemale_mean']/df_sub_fem.loc[:, 'female_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,0,R-squared:,0.0111
Estimator:,PanelOLS,R-squared (Between):,0.2531
No. Observations:,72153,R-squared (Within):,0.0061
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.1782
Time:,11:55:38,Log-likelihood,-1.576e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,161.68
Entities:,23,P-value,0.0000
Avg Obs:,3137.1,Distribution:,"F(5,72082)"
Min Obs:,0.0000,,
Max Obs:,6.138e+04,F-statistic (robust):,7.842e+19


Conflict
...

In [8]:
X = df_sub.loc[:, list_of_features_mean] # raw is ref
y = df_sub.loc[:, 'log_best']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,log_best,R-squared:,0.0011
Estimator:,PanelOLS,R-squared (Between):,-0.0005
No. Observations:,126548,R-squared (Within):,-6.911e-05
Date:,"Thu, Jun 30 2022",R-squared (Overall):,-0.0014
Time:,00:12:07,Log-likelihood,-3617.2
Cov. Estimator:,Clustered,,
,,F-statistic:,8.9986
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(16,126462)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-0.6710


In [7]:
X = df_sub.loc[:, list_of_features_mean] # raw is ref
y = df_sub.loc[:, 'tce']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,tce,R-squared:,0.0056
Estimator:,PanelOLS,R-squared (Between):,-0.0105
No. Observations:,126548,R-squared (Within):,-0.0003
Date:,"Thu, Jun 30 2022",R-squared (Overall):,-0.0065
Time:,00:11:30,Log-likelihood,1.443e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,44.391
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(16,126462)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,0.1840


In [15]:

X = df_sub.loc[:, list_of_features_mean + ['Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'tsce']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,tsce,R-squared:,0.0055
Estimator:,PanelOLS,R-squared (Between):,-0.0058
No. Observations:,126548,R-squared (Within):,-0.0003
Date:,"Thu, Jun 30 2022",R-squared (Overall):,-0.0053
Time:,00:17:32,Log-likelihood,1.17e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,39.208
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(18,126460)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,0.6207


In [13]:

X = df_sub.loc[:, ['religiousGarmentFemale_mean']] # raw is ref
y = df_sub.loc[:, 'tsce']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,tsce,R-squared:,0.0004
Estimator:,PanelOLS,R-squared (Between):,0.0013
No. Observations:,126548,R-squared (Within):,0.0003
Date:,"Thu, Jun 30 2022",R-squared (Overall):,0.0008
Time:,00:16:36,Log-likelihood,1.167e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,47.474
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(1,126477)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,0.8739


In [21]:

df_sub['intTsceFemale'] = df_sub.loc[:,'tsce']  * df_sub.loc[:,'female_mean'] 


X = df_sub.loc[:, ['tsce', 'female_mean', 'intTsceFemale']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3402
Estimator:,PanelOLS,R-squared (Between):,0.8813
No. Observations:,126548,R-squared (Within):,0.3476
Date:,"Thu, Jun 30 2022",R-squared (Overall):,0.3385
Time:,00:26:38,Log-likelihood,-1.742e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,2.173e+04
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(3,126475)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,1710.4


In [19]:
df_sub_fem = df_sub[df_sub['female_mean']>0].copy()

df_sub_fem['intTsceFemale'] = df_sub_fem.loc[:,'tsce']  * df_sub_fem.loc[:,'female_mean'] 

X = df_sub_fem.loc[:,['female_mean', 'tsce', 'intTsceFemale']] # raw is ref
y = df_sub_fem.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.2802
Estimator:,PanelOLS,R-squared (Between):,0.7057
No. Observations:,72153,R-squared (Within):,0.2747
Date:,"Thu, Jun 30 2022",R-squared (Overall):,0.1090
Time:,00:22:59,Log-likelihood,-2.898e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,9355.0
Entities:,23,P-value,0.0000
Avg Obs:,3137.1,Distribution:,"F(3,72084)"
Min Obs:,0.0000,,
Max Obs:,6.138e+04,F-statistic (robust):,1790.7


In [72]:
X = df_sub.loc[:, ['tsce', 'person_mean', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'male_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,male_mean,R-squared:,0.8761
Estimator:,PanelOLS,R-squared (Between):,0.9593
No. Observations:,126548,R-squared (Within):,0.8761
Date:,"Thu, Jun 30 2022",R-squared (Overall):,0.9168
Time:,01:04:24,Log-likelihood,-1.633e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,2.235e+05
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-2.197e+18


In [74]:
X = df_sub.loc[:, ['tsce', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'person_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,person_mean,R-squared:,0.0012
Estimator:,PanelOLS,R-squared (Between):,-0.2378
No. Observations:,126548,R-squared (Within):,-0.0145
Date:,"Thu, Jun 30 2022",R-squared (Overall):,-0.5987
Time:,01:09:26,Log-likelihood,-2.696e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,51.782
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(3,126475)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-2.273e+24


In [77]:
X = df_sub.loc[:, ['tsce', 'person_mean', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'militaryVehicle_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,militaryVehicle_mean,R-squared:,0.0063
Estimator:,PanelOLS,R-squared (Between):,-0.0250
No. Observations:,126548,R-squared (Within):,0.0077
Date:,"Thu, Jun 30 2022",R-squared (Overall):,-0.0267
Time:,01:11:43,Log-likelihood,3.929e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,201.92
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(4,126474)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,3.265e+15


In [25]:
list_of_features_mean

['person_mean',
 'child_mean',
 'male_mean',
 'adult_mean',
 'youth_mean',
 'falgIRQ_mean',
 'female_mean',
 'religiousGarmentFemale_mean',
 'uniformed_mean',
 'firearm_mean',
 'flagUS_mean',
 'bloodedArea_mean',
 'militaryVehicle_mean',
 'hostage_mean',
 'casualty_mean',
 'elderly_mean']

In [50]:
X = df_sub.loc[:, ['female_mean', 'religiousGarmentFemale_mean', 'person_mean', 'male_mean']] # raw is ref
y = df_sub.loc[:, 'Published']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_time=True, cluster_entity=True)

Dep. Variable:,Published,R-squared:,0.0017
Estimator:,PanelOLS,R-squared (Between):,-2.0649
No. Observations:,126548,R-squared (Within):,0.0027
Date:,"Thu, Jun 30 2022",R-squared (Overall):,0.0008
Time:,00:52:35,Log-likelihood,1.852e+05
Cov. Estimator:,Clustered,,
,,F-statistic:,43.099
Entities:,23,P-value,0.0000
Avg Obs:,5502.1,Distribution:,"F(5,126473)"
Min Obs:,2.0000,,
Max Obs:,1.035e+05,F-statistic (robust):,-2.6569


# old

In [22]:

fitted_dict = dict()

for i in list_of_features_mean:

    X = df_sub.loc[:,['log_best', 'Published', 'Submitted']] # raw is ref
    y = df_sub.loc[:, i]

    m = PanelOLS(dependent=y,
                exog=X,
                entity_effects=True,
                time_effects=True)

    fitted_dict[i] = m.fit(cov_type='clustered', cluster_entity=True)

In [24]:
fitted_dict

LinAlgError: Singular matrix

In [28]:
X = df_sub.loc[:,['female_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_entity=True)

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3554
Estimator:,PanelOLS,R-squared (Between):,0.9075
No. Observations:,152188,R-squared (Within):,0.3676
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4309
Time:,10:48:20,Log-likelihood,-7166.9
Cov. Estimator:,Clustered,,
,,F-statistic:,2.097e+04
Entities:,23,P-value,0.0000
Avg Obs:,6616.9,Distribution:,"F(4,152112)"
Min Obs:,2.0000,,
Max Obs:,1.234e+05,F-statistic (robust):,6.135e+21


In [21]:
X = df_sub.loc[:,['female_mean', 'log_best', 'Published', 'Submitted']] # raw is ref
y = df_sub.loc[:, 'religiousGarmentFemale_mean']

m = PanelOLS(dependent=y,
            exog=X,
            entity_effects=True,
            time_effects=True)

m.fit(cov_type='clustered', cluster_entity=True)

Dep. Variable:,religiousGarmentFemale_mean,R-squared:,0.3554
Estimator:,PanelOLS,R-squared (Between):,0.9075
No. Observations:,152188,R-squared (Within):,0.3676
Date:,"Mon, Jun 27 2022",R-squared (Overall):,0.4309
Time:,10:24:07,Log-likelihood,-7166.9
Cov. Estimator:,Clustered,,
,,F-statistic:,2.097e+04
Entities:,23,P-value,0.0000
Avg Obs:,6616.9,Distribution:,"F(4,152112)"
Min Obs:,2.0000,,
Max Obs:,1.234e+05,F-statistic (robust):,6.135e+21
